In [55]:
# Depenencies
import pandas as pd
from sqlalchemy import create_engine, inspect, func
import csv, os
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from flask import jsonify

In [56]:
# Importing csv files
ufo_path = "Resources/ufo_reports_raw.CSV"
ufo_data = pd.read_csv(ufo_path)

# Transformation Phase (Cleaning)

### USA UFO Sightings

In [57]:
ufo_data.head()

,summary,city,state,date_time,shape,duration,stats,report_link,text,posted,city_latitude,city_longitude
0,freaked me out,San Jose,CA,2021-05-06T21:00:00,sphere,2 minutes,Occurred : 5/6/2021 21:00 (Entered as : 05/06...,http://www.nuforc.org/webreports/163/S163041.html,freaked me out I looked up into the night sky ...,2021-05-20T00:00:00,37.338842,-121.889706
1,"One object observed at closer ground distance,...",Somerset,KY,NaN,unknown,2400 hours,Occurred : Reported: 12/21/2019 8:49:14 PM 2...,http://www.nuforc.org/webreports/151/S151813.html,"One object observed at closer ground distance,...",NaN,37.090361,-84.498169
2,UFO contact we made during Desert Shield in th...,NaN,NaN,NaN,NaN,NaN,Occurred : Reported: 5/16/2021 4:55:36 PM 16...,http://www.nuforc.org/webreports/163/S163175.html,UFO contact we made during Desert Shield in th...,NaN,NaN,NaN
3,"Over a course of 5 years, I have seen UFOs, ob...",Germantown,MD,2005-07-06T12:00:00,formation,Years,Occurred : 7/6/1905 12:00 (Entered as : 2014 ...,http://www.nuforc.org/webreports/162/S162867.html,"Over a course of 5 years, I have seen UFOs, ob...",2021-05-20T00:00:00,39.154986,-77.272538
4,"Driving north on US 87 just outside Brady, Tex...",Brady,TX,2020-01-29T19:45:00,light,Lights disappear,Occurred : 1/29/2020 19:45 (Entered as : 01/2...,http://www.nuforc.org/webreports/152/S152953.html,"Driving north on US 87 just outside Brady, Tex...",2020-01-31T00:00:00,31.145100,-99.347800


In [58]:
# Split date_time column
ufo_data[["date","time"]] = ufo_data['date_time'].str.split("T",expand=True)

# Convert date and time columns data type
ufo_data['date'] = pd.to_datetime(ufo_data['date'])
ufo_data['time'] = pd.to_datetime(ufo_data['time'])

In [59]:
ufo_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98353 entries, 0 to 98352
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   summary         98305 non-null  object        
 1   city            98052 non-null  object        
 2   state           92707 non-null  object        
 3   date_time       97027 non-null  object        
 4   shape           94682 non-null  object        
 5   duration        93952 non-null  object        
 6   stats           98316 non-null  object        
 7   report_link     98353 non-null  object        
 8   text            98280 non-null  object        
 9   posted          97027 non-null  object        
 10  city_latitude   80483 non-null  float64       
 11  city_longitude  80483 non-null  float64       
 12  date            97027 non-null  datetime64[ns]
 13  time            97027 non-null  datetime64[ns]
dtypes: datetime64[ns](2), float64(2), object(10)
memory us

In [60]:
#ufo_data.columns

to_drop = ['posted',
            'stats',
            'report_link',
            'date_time',
            'text'
            ]

ufo_data.drop(to_drop, axis=1, inplace=True)

ufo_data.head()

,summary,city,state,shape,duration,city_latitude,city_longitude,date,time
0,freaked me out,San Jose,CA,sphere,2 minutes,37.338842,-121.889706,2021-05-06,2021-12-10 21:00:00
1,"One object observed at closer ground distance,...",Somerset,KY,unknown,2400 hours,37.090361,-84.498169,NaT,NaT
2,UFO contact we made during Desert Shield in th...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
3,"Over a course of 5 years, I have seen UFOs, ob...",Germantown,MD,formation,Years,39.154986,-77.272538,2005-07-06,2021-12-10 12:00:00
4,"Driving north on US 87 just outside Brady, Tex...",Brady,TX,light,Lights disappear,31.145100,-99.347800,2020-01-29,2021-12-10 19:45:00


In [61]:
# Drop UFO Sightings with no city and date
ufo_df = ufo_data.dropna(subset=['city', 'date', 'time'])

ufo_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 96774 entries, 0 to 98352
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   summary         96730 non-null  object        
 1   city            96774 non-null  object        
 2   state           91394 non-null  object        
 3   shape           93309 non-null  object        
 4   duration        92607 non-null  object        
 5   city_latitude   79569 non-null  float64       
 6   city_longitude  79569 non-null  float64       
 7   date            96774 non-null  datetime64[ns]
 8   time            96774 non-null  datetime64[ns]
dtypes: datetime64[ns](2), float64(2), object(5)
memory usage: 7.4+ MB


In [62]:
# Replacing NA values
values = {  'summary': 'observed', 
            'shape':'other',
            'duration':'unknown',
            'text':'observed'
            }
ufo_df2 = ufo_df.fillna(value=values)
ufo_df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96774 entries, 0 to 98352
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   summary         96774 non-null  object        
 1   city            96774 non-null  object        
 2   state           91394 non-null  object        
 3   shape           96774 non-null  object        
 4   duration        96774 non-null  object        
 5   city_latitude   79569 non-null  float64       
 6   city_longitude  79569 non-null  float64       
 7   date            96774 non-null  datetime64[ns]
 8   time            96774 non-null  datetime64[ns]
dtypes: datetime64[ns](2), float64(2), object(5)
memory usage: 7.4+ MB


In [63]:
# Change time column to day or night
def determine_time(time):
    h = time.hour
    return (
        "morning"
        if 5 <= h <= 11
        else "afternoon"
        if 12 <= h <= 17
        else "evening"
        if 18 <= h <= 22
        else "night"
    )

ufo_df2['time'] = ufo_df2.apply(lambda row: determine_time(row[8]), axis=1)

ufo_df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96774 entries, 0 to 98352
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   summary         96774 non-null  object        
 1   city            96774 non-null  object        
 2   state           91394 non-null  object        
 3   shape           96774 non-null  object        
 4   duration        96774 non-null  object        
 5   city_latitude   79569 non-null  float64       
 6   city_longitude  79569 non-null  float64       
 7   date            96774 non-null  datetime64[ns]
 8   time            96774 non-null  object        
dtypes: datetime64[ns](1), float64(2), object(6)
memory usage: 7.4+ MB


In [64]:
ufo_df2.head()

,summary,city,state,shape,duration,city_latitude,city_longitude,date,time
0,freaked me out,San Jose,CA,sphere,2 minutes,37.338842,-121.889706,2021-05-06,evening
3,"Over a course of 5 years, I have seen UFOs, ob...",Germantown,MD,formation,Years,39.154986,-77.272538,2005-07-06,afternoon
4,"Driving north on US 87 just outside Brady, Tex...",Brady,TX,light,Lights disappear,31.145100,-99.347800,2020-01-29,evening
5,meteor or space junk?,San Diego,CA,fireball,2 seconds,32.787229,-117.140268,2020-01-29,evening
6,"Shaped like a triangle, transparent like a lig...",Las Vegas,NV,triangle,30,36.141246,-115.186592,2020-01-28,afternoon


In [65]:
# Re-cast date column into object
ufo_df2['date'] = ufo_df2['date'].dt.strftime("%Y/%m/%d")
ufo_df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96774 entries, 0 to 98352
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   summary         96774 non-null  object 
 1   city            96774 non-null  object 
 2   state           91394 non-null  object 
 3   shape           96774 non-null  object 
 4   duration        96774 non-null  object 
 5   city_latitude   79569 non-null  float64
 6   city_longitude  79569 non-null  float64
 7   date            96774 non-null  object 
 8   time            96774 non-null  object 
dtypes: float64(2), object(7)
memory usage: 7.4+ MB


In [66]:
# Filter data to the past 40 years
# filtered_df = ufo_df2[ufo_df2['date'].dt.year > 1980]
# filtered_df.info()

In [67]:
# Drop UFO sightings with no coordinates 
ufo_df3 = ufo_df2.dropna(subset=['city_latitude', 'city_longitude'])

In [68]:
ufo_df3.reset_index(drop=True)
ufo_df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 79569 entries, 0 to 98352
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   summary         79569 non-null  object 
 1   city            79569 non-null  object 
 2   state           79569 non-null  object 
 3   shape           79569 non-null  object 
 4   duration        79569 non-null  object 
 5   city_latitude   79569 non-null  float64
 6   city_longitude  79569 non-null  float64
 7   date            79569 non-null  object 
 8   time            79569 non-null  object 
dtypes: float64(2), object(7)
memory usage: 6.1+ MB


In [69]:
# Assign Country Code to Index
usa_id = pd.Series([840 for x in range(len(ufo_df3))])
ufo_df3['country_id'] = usa_id
ufo_df3 = ufo_df3.fillna(value=840)

usa_ufo_df = ufo_df3.set_index('country_id')

usa_ufo_df

<ipython-input-69-1f260a7591fa>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ufo_df3['country_id'] = usa_id


,summary,city,state,shape,duration,city_latitude,city_longitude,date,time
country_id,,,,,,,,,
840.0,freaked me out,San Jose,CA,sphere,2 minutes,37.338842,-121.889706,2021/05/06,evening
840.0,"Over a course of 5 years, I have seen UFOs, ob...",Germantown,MD,formation,Years,39.154986,-77.272538,2005/07/06,afternoon
840.0,"Driving north on US 87 just outside Brady, Tex...",Brady,TX,light,Lights disappear,31.145100,-99.347800,2020/01/29,evening
840.0,meteor or space junk?,San Diego,CA,fireball,2 seconds,32.787229,-117.140268,2020/01/29,evening
840.0,"Shaped like a triangle, transparent like a lig...",Las Vegas,NV,triangle,30,36.141246,-115.186592,2020/01/28,afternoon
...,...,...,...,...,...,...,...,...,...
840.0,White bright quick light,St. Petersburg,FL,flash,~2 seconds,27.794515,-82.675160,2021/03/27,night
840.0,It’s really hard to explain... other than at f...,Moses lake,WA,unknown,30,47.190100,-119.307400,2021/03/27,night
840.0,Object in sky above me walking my dog. Came do...,Yorba linda,CA,light,unknown,33.891402,-117.777835,2021/03/27,night


In [70]:
usa_ufo_df.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 79569 entries, 840.0 to 840.0
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   summary         79569 non-null  object 
 1   city            79569 non-null  object 
 2   state           79569 non-null  object 
 3   shape           79569 non-null  object 
 4   duration        79569 non-null  object 
 5   city_latitude   79569 non-null  float64
 6   city_longitude  79569 non-null  float64
 7   date            79569 non-null  object 
 8   time            79569 non-null  object 
dtypes: float64(2), object(7)
memory usage: 6.1+ MB


### Load into PostgreSQL database

In [71]:
connection_string = "postgres:Golfer7!@localhost:5432/ufo_db"
engine = create_engine(f'postgresql://{connection_string}')

In [72]:
# Confirm tables
engine.table_names()

<ipython-input-72-2d97e9123e98>:2: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['usa_ufo']

In [73]:
usa_ufo_df.to_sql(name='usa_ufo', con=engine, if_exists='append', index=True)

### Testing queries

In [74]:
# reflect an existing database into a new model
Base = automap_base()
inspector = inspect(engine)

# reflect the tables
Base.prepare(engine, reflect=True)

# Print all of the classes mapped to the Base
Base.classes.keys()

['usa_ufo']

In [75]:
# Save reference to the table
#usa_table = Base.classes.usa_ufo

# Create our session (link) from Python to the DB
#session = Session(engine)

# sel = [usa_table.report_id , usa_table.city]
#query = session.query(row_to_json(usa_table)).limit(2).all()

#session.close()

# Convert query results into dictionary and jsonify for user
# results = dict(query)
# jsonify(results)

query = engine.execute('SELECT row_to_json(usa_ufo) FROM usa_ufo LIMIT 2').fetchall()

my_list = []

for i in range(len(query)):
    my_list.append(query[i][0])

my_list

[{'report_id': 79570,
  'country_id': 840,
  'summary': 'freaked me out',
  'city': 'San Jose',
  'state': 'CA',
  'shape': 'sphere',
  'duration': '2 minutes',
  'city_latitude': 37.33884202188921,
  'city_longitude': -121.88970568835909,
  'time': 'evening',
  'date': '2021-05-06'},
 {'report_id': 79571,
  'country_id': 840,
  'summary': 'Over a course of 5 years, I have seen UFOs, observed bizarre behavior, and believe I may have directly been affected by it.',
  'city': 'Germantown',
  'state': 'MD',
  'shape': 'formation',
  'duration': 'Years',
  'city_latitude': 39.15498641618498,
  'city_longitude': -77.27253757225458,
  'time': 'afternoon',
  'date': '2005-07-06'}]

In [76]:
import pandas as pd
from sqlalchemy import create_engine
import csv, os

### Other Country UFO Sightings

In [51]:
other_ufo_df = ufo_df2[ufo_df2['state'].isnull()]
other_ufo_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5380 entries, 12 to 98344
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   summary         5380 non-null   object 
 1   city            5380 non-null   object 
 2   state           0 non-null      object 
 3   shape           5380 non-null   object 
 4   duration        5380 non-null   object 
 5   city_latitude   0 non-null      float64
 6   city_longitude  0 non-null      float64
 7   date            5380 non-null   object 
 8   time            5380 non-null   object 
dtypes: float64(2), object(7)
memory usage: 420.3+ KB


In [52]:
other_ufo_df

,summary,city,state,shape,duration,city_latitude,city_longitude,date,time
12,A series of soft white lights were traveling i...,Chennai (India),NaN,light,10-15 minutes,NaN,NaN,2020/01/10,morning
84,Big UFO hovering over a house,Comalcalco (Mexico),NaN,other,4:00,NaN,NaN,1980/06/01,afternoon
88,30+ orange balls playing in Australia,Hobart (Australia),NaN,fireball,5 minutes,NaN,NaN,1982/03/16,evening
96,Simple triangle 40/50 yards across. Blue soft ...,Hahn A B (Germany),NaN,triangle,7 minutes,NaN,NaN,1988/06/01,night
103,"Incredible bright light flooding the room, acc...","Chipping (small village, Lancashire)(UK/England)",NaN,other,3-30 minutes,NaN,NaN,1990/06/01,night
...,...,...,...,...,...,...,...,...,...
98239,Boomarang like shaped with five lights made a ...,Firozabad (India),NaN,changing,6 minutes,NaN,NaN,2021/04/22,evening
98256,Strange perfect straight lights that kept form...,Rugby (UK/England),NaN,formation,30 seconds,NaN,NaN,2021/03/12,morning
98288,We took this picture of my family sitting on t...,EK Balam (Yucatan)(Mexico),NaN,sphere,Instant,NaN,NaN,2021/03/17,morning
98307,"Blue Orbs floating around Fagradals Mountain, ...",Iceland,NaN,other,15,NaN,NaN,2021/03/21,night


In [53]:
# Split city column to obtain country
location_df = other_ufo_df['city'].str.split( '(' , expand=True, n=2)

df = location_df.rename(columns={0:'city',1:'country'})
# df['country'].fillna(df['city'], inplace=True)

# Drop Columns with no city and country combination
df2 = df.dropna(subset=['country'])
df3 = df2.drop(df2[df2[2].notnull()].index)
df3.drop(2, axis=1, inplace=True)
df3

,city,country
12,Chennai,India)
84,Comalcalco,Mexico)
88,Hobart,Australia)
96,Hahn A B,Germany)
123,Stoke on Trent,UK/England)
...,...,...
98126,Northwich,UK/England)
98206,Tamworth,UK/England)
98239,Firozabad,India)
98256,Rugby,UK/England)


In [54]:
df3['country'] = df3['country'].str.replace( ')','', regex=True)
df3

,city,country
12,Chennai,India
84,Comalcalco,Mexico
88,Hobart,Australia
96,Hahn A B,Germany
123,Stoke on Trent,UK/England
...,...,...
98126,Northwich,UK/England
98206,Tamworth,UK/England
98239,Firozabad,India
98256,Rugby,UK/England
